In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torchtext.vocab import build_vocab_from_iterator
# from torchtext.data.utils import get_tokenizer
import spacy
import pickle
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

In [5]:
df = pd.read_excel("Product Matching Phase 1\Product Matching Phase 1\Product Matching Dataset.xlsx", 'Dataset')

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Ahmed Rabie\AppData\Local\Temp\ipykernel_9056\2690156247.py:1: SyntaxWarning: invalid escape sequence '\P'
  df = pd.read_excel("Product Matching Phase 1\Product Matching Phase 1\Product Matching Dataset.xlsx", 'Dataset')


In [6]:
df.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,ESTOHALT 40 MG 14 CAP,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5


In [7]:
df.duplicated().sum()

33990

In [8]:
df_cleaned = df.drop_duplicates()

In [9]:
df_cleaned.head()

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,ESTOHALT 40 MG 14 CAP,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
7,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 1 شريط * 14 كبسولة,56.5
8,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم,56.5
9,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 كبسول س ج,56.5


In [10]:
df_cleaned.size

198288

In [11]:
label_encoder = LabelEncoder()
df_cleaned["label"] = label_encoder.fit_transform(df_cleaned["marketplace_product_name_ar"])

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_cleaned["seller_item_name"].tolist(), df_cleaned["label"].tolist(), test_size=0.2, random_state=42
)

test_texts, val_texts, test_labels, val_labels = train_test_split(
    val_texts, val_labels, test_size=0.5, random_state=42
)

num_classes = len(label_encoder.classes_)   

C:\Users\Ahmed Rabie\AppData\Local\Temp\ipykernel_9056\2344435659.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["label"] = label_encoder.fit_transform(df_cleaned["marketplace_product_name_ar"])


In [12]:
nlp = spacy.blank("ar")

In [13]:
def arabic_tokenizer(text):
    return [token.text for token in nlp(text)]

def yield_tokens(texts):
    for text in texts:
        yield arabic_tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_texts), specials=["<pad>", "<unk>"])
vocab.set_default_index(vocab["<unk>"])

def text_to_tensor(text, max_len=50):
    tokens = arabic_tokenizer(text)[:max_len]
    token_ids = [vocab[token] for token in tokens]
    return torch.tensor(token_ids + [vocab["<pad>"]] * (max_len - len(token_ids)), dtype=torch.long)

train_inputs = torch.stack([text_to_tensor(text) for text in train_texts])
val_inputs = torch.stack([text_to_tensor(text) for text in val_texts])
test_inputs = torch.stack([text_to_tensor(text) for text in test_texts])
train_labels = torch.tensor(train_labels, dtype=torch.long)
val_labels = torch.tensor(val_labels, dtype=torch.long)
test_labels = torch.tensor(test_labels, dtype=torch.long)

In [14]:
class ArabicTextDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {"input_ids": self.inputs[idx], "label": self.labels[idx]}

In [15]:
batch_size = 32
train_dataset = ArabicTextDataset(train_inputs, train_labels)
val_dataset = ArabicTextDataset(val_inputs, val_labels)
test_dataset = ArabicTextDataset(test_inputs, test_labels)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [16]:
class SimpleTransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, hidden_dim, num_layers, num_classes):
        super(SimpleTransformerClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.pos_embedding = nn.Parameter(torch.zeros(1, 50, embed_dim))

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Linear(embed_dim, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, input_ids):
        x = self.embedding(input_ids) + self.pos_embedding[:, :input_ids.size(1), :]
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)  
        x = self.dropout(x)
        return self.fc(x)

embed_dim = 128   
num_heads = 4     
hidden_dim = 256  
num_layers = 2    

device = torch.device("cpu")

model = SimpleTransformerClassifier(
    vocab_size=len(vocab),
    embed_dim=embed_dim,
    num_heads=num_heads,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    num_classes=num_classes
).to(device)

C:\Users\Ahmed Rabie\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-4)

def train(model, dataloader, optimizer, loss_fn, device):
    model.train()
    total_loss, correct = 0, 0

    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    return total_loss / len(dataloader), correct / len(dataloader.dataset)


def evaluate(model, dataloader, loss_fn, device):
    model.eval()
    total_loss, correct = 0, 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids)
            loss = loss_fn(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

    return total_loss / len(dataloader), correct / len(dataloader.dataset)


In [17]:
device

device(type='cpu')

In [21]:
epochs = 5

for epoch in range(epochs):
    train_loss, train_acc = train(model, train_loader, optimizer, loss_fn, device)
    val_loss, val_acc = evaluate(model, val_loader, loss_fn, device)

    print(f"Epoch {epoch+1}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")


Epoch 1:
  Train Loss: 0.2298, Train Acc: 0.9660
  Val Loss: 0.4074, Val Acc: 0.9357
Epoch 2:
  Train Loss: 0.2075, Train Acc: 0.9687
  Val Loss: 0.4158, Val Acc: 0.9373
Epoch 3:
  Train Loss: 0.1878, Train Acc: 0.9711
  Val Loss: 0.3891, Val Acc: 0.9369
Epoch 4:
  Train Loss: 0.1708, Train Acc: 0.9736
  Val Loss: 0.3963, Val Acc: 0.9387
Epoch 5:
  Train Loss: 0.1567, Train Acc: 0.9764
  Val Loss: 0.4175, Val Acc: 0.9351


In [ ]:
test_loss, test_acc = evaluate(model, test_loader, loss_fn, device)
print(f"Test Loss: {val_loss:.4f}, Test Acc: {val_acc:.4f}")

In [52]:
device = 'cpu'
    
model = SimpleTransformerClassifier(
        vocab_size=len(vocab),
        embed_dim=embed_dim,
        num_heads=num_heads,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=num_classes
    ).to(device)
model.load_state_dict(torch.load("simple_transformer_arabic.pth", map_location=device))
model.eval()
    
with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

model.eval()
all_probs = []
all_labels = []

C:\Users\Ahmed Rabie\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [53]:
with torch.no_grad():
    for batch in val_loader:
        inputs, labels = batch["input_ids"].to(device), batch["label"].to(device)
        outputs = model(inputs)
        probabilities = F.softmax(outputs, dim=1)   
        
        all_probs.append(probabilities.cpu().numpy())  
        all_labels.append(labels.cpu().numpy()) 


all_probs = np.concatenate(all_probs)
all_labels = np.concatenate(all_labels)


In [54]:
thresholds = np.linspace(0.1, 0.9, 9)
best_threshold = 0
best_acc = 0   

In [ ]:
for threshold in thresholds:
    predictions = np.argmax(all_probs, axis=1)   
    
    confident_predictions = np.where(np.max(all_probs, axis=1) > threshold, predictions, -1)

    valid_indices = confident_predictions != -1
    filtered_preds = confident_predictions[valid_indices]
    filtered_labels = all_labels[valid_indices]

    if len(filtered_preds) > 0:   
        acc = accuracy_score(filtered_labels, filtered_preds)   
        
        if acc > best_acc:
            best_acc = acc
            best_threshold = threshold

print(f"✅ Best Threshold: {best_threshold:.2f} (acc-score: {best_acc:.4f})")

✅ Best Threshold: 0.90 (acc-score: 0.9972)


In [26]:
torch.save(model.state_dict(), "simple_transformer_arabic.pth")

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

In [67]:
def classify_text(text, model, label_encoder, device, lookup_df, lookup_column, sku_column, best_threshold):
    input_tensor = text_to_tensor(text).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(input_tensor)
        probabilities = F.softmax(outputs, dim=1)   
        confidence, predicted_class = torch.max(probabilities, dim=1)  
        
        if probabilities[0][predicted_class] < best_threshold:
            return "Uncertain", confidence.item() * 100
    
    predicted_label = label_encoder.inverse_transform([predicted_class.item()])[0]
    confidence_score = confidence.item() * 100    
    
    # Lookup SKU value
    sku_value = lookup_df.loc[lookup_df[lookup_column] == predicted_label, sku_column]
    if not sku_value.empty:
        return sku_value.values[0], confidence_score
    return predicted_label, confidence_score

In [68]:
def classify_excel(file_path, text_column, lookup_file, lookup_column, output_path, best_threshold = 0.9, sku_column = 'sku'):
    device = 'cpu'
    
    # Load model
    model = SimpleTransformerClassifier(
        vocab_size=len(vocab),
        embed_dim=embed_dim,
        num_heads=num_heads,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=num_classes
    ).to(device)
    model.load_state_dict(torch.load("simple_transformer_arabic.pth", map_location=device))
    model.eval()
    
    # Load label encoder
    with open("label_encoder.pkl", "rb") as f:
        label_encoder = pickle.load(f)
    
    # Load Excel files
    df = pd.read_excel(file_path)
    lookup_df = pd.read_excel(lookup_file)
    
    # Ensure text column exists
    if text_column not in df.columns:
        raise ValueError(f"Column '{text_column}' not found in the Excel file.")
    
    # Apply classification
    predictions = df[text_column].apply(lambda text: classify_text(str(text), model, label_encoder, device, lookup_df, lookup_column, sku_column, best_threshold))
    df['SKU_Value'], df['Confidence_Score'] = zip(*predictions)
    
    # Save output
    df.to_excel(output_path, index=False)
    print(f"Predictions saved to {output_path}")

In [69]:
%%time
classify_excel('Product Matching Phase 1/Product Matching Phase 1/test.xlsx', 'seller_item_name','Product Matching Phase 1\Product Matching Phase 1\Product Matching Dataset.xlsx', 'product_name_ar' ,  'Product Matching Phase 1\Product Matching Phase 1\output.xlsx')

<unknown>:1: SyntaxWarning: invalid escape sequence '\P'
<unknown>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Ahmed Rabie\AppData\Roaming\Python\Python312\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Predictions saved to Product Matching Phase 1\Product Matching Phase 1\output.xlsx
CPU times: total: 42min 30s
Wall time: 4min 18s


In [65]:
tmp = pd.read_excel('Product Matching Phase 1\Product Matching Phase 1\output.xlsx')

<>:1: SyntaxWarning: invalid escape sequence '\P'
<>:1: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Ahmed Rabie\AppData\Local\Temp\ipykernel_9056\3244577068.py:1: SyntaxWarning: invalid escape sequence '\P'
  tmp = pd.read_excel('Product Matching Phase 1\Product Matching Phase 1\output.xlsx')


In [66]:
c = 0

for txt in tmp['Predicted_Class']:
    if txt != 'Uncertain':
        c += 1

print(c / len(tmp['Predicted_Class']))

0.8082740958809028
